In [ ]:
!pip install requests

In [ ]:

import requests
import random
import time
from datetime import datetime, timedelta

API_KEY = "82C72VKGF0TR7PJJ"         # Write API Key
READ_API_KEY = "0AUWKIAQO4AINI71"    # Read API Key
CHANNEL_ID = 2894456                 # ThingSpeak Channel ID
THING_SPEAK_URL = "https://api.thingspeak.com/update"


In [ ]:

def generate_sensor_data():
    return {
        "field1": round(random.uniform(-50, 50), 1),
        "field2": round(random.uniform(0, 100), 1),
        "field3": round(random.uniform(300, 2000), 1)
    }

def send_data_to_thingspeak():
    for _ in range(5):
        data = generate_sensor_data()
        payload = {
            "api_key": API_KEY,
            **data
        }
        print(f"Sending: {payload}")
        response = requests.post(THING_SPEAK_URL, data=payload)
        print(f"Status: {response.status_code} | Entry ID: {response.text}")
        time.sleep(15)

send_data_to_thingspeak()


In [ ]:

def get_latest_values():
    url = f"https://api.thingspeak.com/channels/{CHANNEL_ID}/feeds.json?api_key={READ_API_KEY}&results=1"
    response = requests.get(url)
    if response.status_code == 200:
        latest = response.json()['feeds'][0]
        print("📊 Latest Sensor Data:")
        print(f"Timestamp: {latest['created_at']}")
        print(f"Temperature: {latest['field1']} °C")
        print(f"Humidity: {latest['field2']} %")
        print(f"CO₂: {latest['field3']} ppm")
    else:
        print("❌ Error fetching data")

get_latest_values()


In [ ]:

def get_last_5_hours(sensor_field="field3"):
    url = f"https://api.thingspeak.com/channels/{CHANNEL_ID}/feeds.json?api_key={READ_API_KEY}&results=100"
    response = requests.get(url)
    
    if response.status_code == 200:
        feeds = response.json().get('feeds', [])
        print(f"📈 {sensor_field} Data in Last 5 Hours:")
        
        for feed in feeds:
            value = feed.get(sensor_field)
            if value is not None:
                timestamp = datetime.strptime(feed['created_at'], "%Y-%m-%dT%H:%M:%SZ")
                if datetime.utcnow() - timestamp <= timedelta(hours=5):
                    print(f"{timestamp} => {sensor_field}: {value}")
    else:
        print("❌ Error fetching data. Status:", response.status_code)

# Example:
get_last_5_hours("field3")
